In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
import numpy as np
# import altair as alt
import copy

warnings.filterwarnings('ignore')
# _ = alt.data_transformers.disable_max_rows()

In [2]:
df = pd.read_json('info.json')

In [3]:
df.head()

,name,price,basics,features
0,2018 Ford Escape Titanium,25411,"{'exterior_color': 'Blue Metallic', 'interior_...","{'heated_seats': True, 'heated_steering_wheel'..."
1,2013 Jaguar XJ L Portfolio,28977,"{'exterior_color': 'Ultimate Black', 'interior...","{'heated_seats': False, 'heated_steering_wheel..."
2,2020 Volvo S60 T5 Momentum,32900,"{'exterior_color': 'Osmium Gray Metallic', 'in...","{'heated_seats': False, 'heated_steering_wheel..."
3,2021 Toyota Highlander Hybrid Platinum,49846,"{'exterior_color': 'Black', 'interior_color': ...","{'heated_seats': True, 'heated_steering_wheel'..."
4,2020 Lexus NX 300 F Sport,38689,"{'exterior_color': 'Atomic Silver', 'interior_...","{'heated_seats': False, 'heated_steering_wheel..."


## Data Preprocessing

### Expand dictionary data, extract year

In [4]:
df_expand = pd.concat([df.drop(['basics', 'features'], axis=1), 
                       df['basics'].apply(pd.Series), df['features'].apply(pd.Series)], axis=1)
df_expand['year'] = df_expand['name'].apply(lambda x: int(re.search(r'^\d+', x).group(0)))

In [5]:
df_expand.columns

Index(['name', 'price', 'exterior_color', 'interior_color', 'drive_train',
       'mpg', 'fuel_type', 'transmission', 'engine', 'mileage', 'heated_seats',
       'heated_steering_wheel', 'nav_sys', 'remote_start', 'carplay',
       'bluetooth', 'brake_assist', 'blind_spot_monitor', 'year'],
      dtype='object')

### Set up filter for safety, comfort, multimedia configs

In [6]:
def filter(df, feature_list):
    df_filtered = copy.deepcopy(df)
    for feature in feature_list:
        df_filtered = df_filtered[df_filtered[feature] == True]
    return df_filtered

In [7]:
safety_list = ['brake_assist', 'blind_spot_monitor']
comfort_list = ['heated_seats', 'heated_steering_wheel', 'remote_start']
multimedia_list = ['nav_sys', 'carplay', 'bluetooth']

In [8]:
filter(filter(filter(df_expand, safety_list), comfort_list), multimedia_list)

,name,price,exterior_color,interior_color,drive_train,mpg,fuel_type,transmission,engine,mileage,heated_seats,heated_steering_wheel,nav_sys,remote_start,carplay,bluetooth,brake_assist,blind_spot_monitor,year
0,2018 Ford Escape Titanium,25411,Blue Metallic,Charcoal Black,Front-wheel Drive,21–28,Gasoline,6-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,47114,True,True,True,True,True,True,True,True,2018
18,2018 Ford Escape Titanium,25411,Blue Metallic,Charcoal Black,Front-wheel Drive,21–28,Gasoline,6-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,47114,True,True,True,True,True,True,True,True,2018
20,2020 Ford Edge SEL,30997,Magnetic Metallic,Ebony,All-wheel Drive,21–28,Gasoline,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,27075,True,True,True,True,True,True,True,True,2020
31,2020 Nissan Murano SL,40750,Brilliant Silver Metallic,Graphite,All-wheel Drive,20–28,Gasoline,Automatic CVT,3.5L V6 24V MPFI DOHC,13566,True,True,True,True,True,True,True,True,2020
142,2018 Volvo XC60 T6 Momentum,40798,Onyx Black Metallic,Blond,All-wheel Drive,21–27,Gasoline,8-Speed Automatic,2.0L I4 16V GDI DOHC,19730,True,True,True,True,True,True,True,True,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9018,2018 Ford Explorer Sport,44491,Magnetic Metallic,Ebony Black,Four-wheel Drive,16–22,Gasoline,6-Speed Automatic with Select-Shift,V6 Cylinder Engine,17865,True,True,True,True,True,True,True,True,2018
9169,2018 Chevrolet Suburban LT,50277,WHITE,Jet Black,Four-wheel Drive,15–22,E85 Flex Fuel,AUTOMATIC,8 Cylinder Engine,55245,True,True,True,True,True,True,True,True,2018
9181,2018 Jeep Grand Cherokee Limited,32205,Bright White Clearcoat,Light Frost Beige / Black,Rear-wheel Drive,19–26,Gasoline,8-Speed Automatic,3.6L V6 24V MPFI DOHC,24855,True,True,True,True,True,True,True,True,2018
9193,2018 Ford Explorer Platinum,40590,Platinum,Medium Soft Ceramic,Four-wheel Drive,16–22,Gasoline,6-Speed Automatic with Select-Shift,V6 Cylinder Engine,26623,True,True,True,True,True,True,True,True,2018


### Simple mapping for drivetrain, fuel type

In [9]:
df_expand.drive_train.unique()

array(['Front-wheel Drive', 'All-wheel Drive', 'Four-wheel Drive', 'AWD',
       'Rear-wheel Drive', '4WD', 'FWD', 'RWD', '–', 'Four Wheel Drive'],
      dtype=object)

In [10]:
drive_train_list = ['FWD', 'AWD', 'AWD', 'AWD', 'RWD', 'AWD', 'FWD', 'RWD', 'unknown', 'AWD']
df_expand['drive_train_simple'] = df_expand['drive_train'].map(dict(zip(df_expand.drive_train.unique(), drive_train_list)))

In [11]:
df_expand.fuel_type.unique()

array(['Gasoline', 'Hybrid', 'Electric', 'E85 Flex Fuel', 'Diesel', '–',
       'Flexible Fuel', 'Gasoline/Mild Electric Hybrid', 'Gasoline Fuel'],
      dtype=object)

In [12]:
fuel_type_list = ['Gasoline', 'Hybrid', 'Electric', 'Flexible Fuel', 'Diesel', '', 'Flexible Fuel', 'Hybrid', 'Gasoline']
df_expand['fuel_type_simple'] = df_expand['fuel_type'].map(dict(zip(df_expand.fuel_type.unique(), fuel_type_list)))

### Extracting emission, cylinder, turbo, average mpg, transmission

In [13]:
df_expand['turbo'] = df_expand['engine'].apply(lambda x: 'turbo' in x.lower())

In [14]:
def extract_emission_cylinder(engine):
    if engine == 'Electric':
        return (0.0, 'E')
    else:
        if re.search(r'\d.(\d *L|\d$)', engine):
            em = float(re.search(r'\d.(\d *L|\d$)', engine).group(0).replace('L', '').strip())
        else:
            em = -1.0
        if re.search(r'[V|I|H]-*\d{1,2}', engine):
            cy = re.search('[V|I|H]-*\d{1,2}', engine).group(0).replace('-', '')
        else:
            cy = ''
    return (em, cy)

In [15]:
df_expand['emission'] = df_expand.apply(lambda x: extract_emission_cylinder(x['engine'])[0], axis=1)
df_expand['cylinder'] = df_expand.apply(lambda x: extract_emission_cylinder(x['engine'])[1], axis=1)

In [16]:
def get_average_mpg(range_mpg, fuel_type):
    if fuel_type == 'Electric':
        return 0.0
    elif range_mpg == 'Transmission':
        return -1.0
    else:
        if '–' in range_mpg:
            mpg_min = int(range_mpg[:range_mpg.index('–')])
            mpg_max = int(range_mpg[range_mpg.index('–') + 1:])
            if mpg_min > mpg_max:
                mpg_min = mpg_max
            return (mpg_min + mpg_max) / 2
        else:
            return float(range_mpg)

In [17]:
df_expand['average_mpg'] = df_expand.apply(lambda x: get_average_mpg(x['mpg'], x['fuel_type_simple']), axis=1)

In [18]:
def extract_transmission(transmission, fuel_type):
    if fuel_type == 'Electric':
        return 0
    else:
        if re.search(r'\d{1,2}-* *(speed|spd)', transmission.lower()):
            return int(re.search(r'\d{1,2}-* *(speed|spd)', transmission.lower()).group(0)
                       .replace('speed', '').replace('spd', '').replace('-', '').strip())
        else:
            return -1

In [19]:
df_expand['transmission_simple'] = df_expand.apply(lambda x:extract_transmission(x['transmission'], x['fuel_type_simple']), axis=1)

In [20]:
performance_list = ['drive_train_simple', 'fuel_type_simple', 'average_mpg', 'emission', 'cylinder', 'transmission_simple']

In [21]:
df_expand.head()

,name,price,exterior_color,interior_color,drive_train,mpg,fuel_type,transmission,engine,mileage,...,brake_assist,blind_spot_monitor,year,drive_train_simple,fuel_type_simple,turbo,emission,cylinder,average_mpg,transmission_simple
0,2018 Ford Escape Titanium,25411,Blue Metallic,Charcoal Black,Front-wheel Drive,21–28,Gasoline,6-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,47114,...,True,True,2018,FWD,Gasoline,True,2.0,I4,24.5,6
1,2013 Jaguar XJ L Portfolio,28977,Ultimate Black,Jet,All-wheel Drive,16–24,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Supercharged,31208,...,True,False,2013,AWD,Gasoline,False,3.0,V6,20.0,8
2,2020 Volvo S60 T5 Momentum,32900,Osmium Gray Metallic,Charcoal,Front-wheel Drive,23–34,Gasoline,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,20077,...,True,False,2020,FWD,Gasoline,True,2.0,I4,28.5,8
3,2021 Toyota Highlander Hybrid Platinum,49846,Black,–,All-wheel Drive,35–34,Hybrid,Automatic CVT,2.5L I4 16V PDI DOHC Hybrid,35335,...,True,False,2021,AWD,Hybrid,False,2.5,I4,34.0,-1
4,2020 Lexus NX 300 F Sport,38689,Atomic Silver,Red,Front-wheel Drive,22–28,Gasoline,6-Speed Automatic,2.0L I4 16V PDI DOHC Turbo,26583,...,True,False,2020,FWD,Gasoline,True,2.0,I4,25.0,6


## Simple plots

In [22]:
# alt.Chart(df_expand).mark_bar(color='green').transform_filter(
#     alt.datum['price'] <= 120000
# ).encode(
#     x=alt.X('price:Q', bin=alt.Bin(maxbins=20)),
#     y=alt.Y('count():Q')
# )

In [23]:
def price_level(price):
    if price > 60000:
        return 'expensive'
    elif price <=60000 and price > 20000:
        return 'moderate'
    else:
        return 'cheap'

In [24]:
df_expand['price_level'] = df_expand['price'].apply(price_level)

In [25]:
# alt.Chart(df_expand).transform_filter(
#     (alt.datum['price'] <= 120000 & alt.datum['drive_train_simple'] != 'none' & alt.datum['year'] > 2005)
# ).mark_bar().encode(
#     x=alt.X('year:N'),
#     y=alt.Y('count():Q'),
#     color=alt.Color('drive_train_simple:N')
# )

In [26]:
len(df_expand[df_expand.fuel_type_simple == 'Electric']) /9204

0.015428074750108649

In [27]:
df_expand.price.mean()

39279.23468057367

In [28]:
df_expand.reset_index(inplace=True)
df_expand = df_expand.rename(columns = {'index':'id', 'name':'contents'})
df_expand.head()

,id,contents,price,exterior_color,interior_color,drive_train,mpg,fuel_type,transmission,engine,...,blind_spot_monitor,year,drive_train_simple,fuel_type_simple,turbo,emission,cylinder,average_mpg,transmission_simple,price_level
0,0,2018 Ford Escape Titanium,25411,Blue Metallic,Charcoal Black,Front-wheel Drive,21–28,Gasoline,6-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,...,True,2018,FWD,Gasoline,True,2.0,I4,24.5,6,moderate
1,1,2013 Jaguar XJ L Portfolio,28977,Ultimate Black,Jet,All-wheel Drive,16–24,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Supercharged,...,False,2013,AWD,Gasoline,False,3.0,V6,20.0,8,moderate
2,2,2020 Volvo S60 T5 Momentum,32900,Osmium Gray Metallic,Charcoal,Front-wheel Drive,23–34,Gasoline,8-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,...,False,2020,FWD,Gasoline,True,2.0,I4,28.5,8,moderate
3,3,2021 Toyota Highlander Hybrid Platinum,49846,Black,–,All-wheel Drive,35–34,Hybrid,Automatic CVT,2.5L I4 16V PDI DOHC Hybrid,...,False,2021,AWD,Hybrid,False,2.5,I4,34.0,-1,moderate
4,4,2020 Lexus NX 300 F Sport,38689,Atomic Silver,Red,Front-wheel Drive,22–28,Gasoline,6-Speed Automatic,2.0L I4 16V PDI DOHC Turbo,...,False,2020,FWD,Gasoline,True,2.0,I4,25.0,6,moderate


In [29]:
df_expand.to_json("data.json", orient='records', lines=True)

## Produce single string form for baseline

In [30]:
df_expand_long = pd.concat([df.drop(['basics', 'features'], axis=1), 
                       df['basics'].apply(pd.Series), df['features'].apply(pd.Series)], axis=1)

In [31]:
config_list = safety_list + comfort_list + multimedia_list

In [32]:
for conf in config_list:
    df_expand_long[conf] = df_expand_long[conf].apply(lambda x: conf if x else 'no ' + conf)

In [33]:
df_expand_long['price'] = df_expand_long['price'].apply(str)
df_expand_long['mileage'] = df_expand_long['mileage'].apply(str)
df_expand_long['content'] = df_expand_long.apply(', '.join, axis=1)

In [34]:
df_expand_long[['content']].to_csv('joined.csv', index_label='docid')